In [1]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 4.8 MB/s eta 0:00:0000:0100:01m


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
#cv
from sklearn.model_selection import cross_val_score


train = pd.read_csv('wineq_train.csv')

train.head()    


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Split the data into X and y
X = train.drop(columns=['quality'])
y = train['quality']

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create data pools
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Initialize the model
model = CatBoostClassifier(loss_function='MultiClass', random_seed=42, verbose=0)

# Define the parameter grid
param_grid = {
    'iterations': [500, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 64, 128, 256]
}

# Initialize StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='neg_root_mean_squared_error', verbose=2, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Initialize the model with the best parameters
best_model = CatBoostClassifier(**best_params, loss_function='MultiClass', random_seed=42, verbose=200)

# Fit the model with the best parameters
best_model.fit(train_pool, eval_set=test_pool)

# Feature selection using manual LOFO
def calculate_lofo_importance(X, y, model, cv):
    baseline_score = -mean_squared_error(y, model.predict(X), squared=False)
    feature_importance = {}
    
    for feature in tqdm(X.columns, desc="Calculating LOFO importance"):
        X_loo = X.drop(columns=[feature])
        scores = []
        
        for train_idx, val_idx in cv.split(X_loo, y):
            X_train_cv, X_val_cv = X_loo.iloc[train_idx], X_loo.iloc[val_idx]
            y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]
            
            model.fit(X_train_cv, y_train_cv)
            y_pred_cv = model.predict(X_val_cv)
            score = -mean_squared_error(y_val_cv, y_pred_cv, squared=False)
            scores.append(score)
        
        feature_importance[feature] = baseline_score - np.mean(scores)
    
    importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['importance'])
    importance_df = importance_df.sort_values(by='importance', ascending=False)
    
    return importance_df

# Calculate LOFO importance
importance_df = calculate_lofo_importance(X_train, y_train, best_model, cv)

# Select features with positive importance
selected_features = importance_df[importance_df["importance"] > 0].index.tolist()
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Create new data pools with selected features
train_pool_selected = Pool(X_train_selected, y_train)
test_pool_selected = Pool(X_test_selected, y_test)

# Fit the model again with the selected features
best_model.fit(train_pool_selected, eval_set=test_pool_selected)

# Predict and evaluate the model
y_pred = best_model.predict(X_test_selected)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# You can add more evaluation metrics here
print("RMSE:", rmse)
print("Classification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 480 candidates, totalling 2400 fits


/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=   1.3s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   1.3s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=   1.3s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   1.3s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   1.4s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.05; total time=   1.4s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   1.4s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.01; total time=   1.5s
[CV] END border_count=32, depth=4, iterations=500, l2_leaf_reg=1, learning_rate=0.1; total time=   1.3s
[CV] END border_count=32, depth=4, iterations=500, l2_le

/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


499:	learn: 0.2503945	test: 0.8315067	best: 0.8315067 (499)	total: 2.81s	remaining: 0us

bestTest = 0.8315067111
bestIteration = 499



...

200:	learn: 0.5343951	total: 1.1s	remaining: 1.64s
400:	learn: 0.2879869	total: 2.35s	remaining: 581ms


/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
Calculating LOFO importance: 100%|██████████| 11/11 [02:42<00:00, 14.79s/it]

499:	learn: 0.2219707	total: 2.88s	remaining: 0us


0:	learn: 1.8664119	test: 1.8712757	best: 1.8712757 (0)	total: 4.37ms	remaining: 2.18s
200:	learn: 0.5722594	test: 0.9060651	best: 0.9060651 (200)	total: 1.1s	remaining: 1.63s
400:	learn: 0.3216893	test: 0.8458424	best: 0.8458424 (400)	total: 2.33s	remaining: 575ms
499:	learn: 0.2560347	test: 0.8358186	best: 0.8355334 (495)	total: 2.93s	remaining: 0us

bestTest = 0.835533401
bestIteration = 495

Shrink model to first 496 iterations.
RMSE: 0.6667792697696852
Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         3
           4       0.50      0.18      0.26        28
           5       0.72      0.69      0.70       238
           6       0.65      0.80      0.72       313
           7       0.70      0.58      0.64       127
           8       0.73      0.38      0.50        29
           9       0.00      0.00      0.00         2

    accuracy                           0.68       740
   macro avg       0.47   

/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kaanhacihaliloglu/padova/pdv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/kaanhacihaliloglu/padova

In [11]:
X_train_selected = X[selected_features]
#X_test_selected = X_test[selected_features]

# Create new data pools with selected features
train_pool_selected = Pool(X_train_selected, y)
#test_pool_selected = Pool(X_test_selected, y_test)

# Fit the model again with the selected features
best_model.fit(train_pool_selected)

val = pd.read_csv('../wineq_validation.csv')
X_val = val[selected_features]

y_val = best_model.predict(X_val)

#write to txt 
with open('wineq_predictionscb.txt', 'w') as f:
    for item in y_val:
        f.write("%s\n" % item)


0:	learn: 1.8649334	total: 41.2ms	remaining: 20.5s
200:	learn: 0.5834450	total: 1.39s	remaining: 2.07s
400:	learn: 0.3466127	total: 2.65s	remaining: 655ms
499:	learn: 0.2762604	total: 3.27s	remaining: 0us


In [10]:
best_params

{'border_count': 128,
 'depth': 8,
 'iterations': 500,
 'l2_leaf_reg': 1,
 'learning_rate': 0.05}